In [109]:
import time
import arrow
#import delorean
from datetime import datetime as dt 
import pandas as pd

In [2]:
from enum import Enum
class Token(Enum):
    NUM=0
    PLUS=1
    MINUS=2
    DAY=3
    WEEK=4
    MONTH=5
    END=6

In [190]:


class DateTranslator:
    def __init__(self, init_date=None):
        if init_date is None:
            self.data = arrow.now()
        else:
            self.data = dt.strptime(init_date, "%Y%m%d")
            self.data = arrow.get(self.data)

  
    def translate_date(self, date_tokens):
        #return a string with the translated date in format yyyymmdd
        #print(date_tokens)
        sign = 0
        time = 0
        for i in date_tokens:
            temp = i
            if temp[0] == Token.MINUS:
                sign = -1
            if temp[0] == Token.PLUS:
                sign = 1
            if temp[0] == Token.NUM:
                time += temp[1] 
            if temp[0] == Token.DAY:
                dat = sign*time
                self.data = self.data.shift(days=dat) 
                time = 0
            if temp[0] == Token.WEEK:
                dat = sign*time
                self.data = self.data.shift(weeks=dat) 
                time = 0
            if temp[0] == Token.MONTH:
                dat = sign*time
                self.data = self.data.shift(months=dat) 
                time = 0
            
                

    def tokenizer(self, date_phrase):
        #return list of tokens
        tokens=[]
        token_map={'מחר':((Token.PLUS,0),(Token.NUM,1),(Token.DAY,0)),
                  'אתמול':((Token.MINUS,0),(Token.NUM,1),(Token.DAY,0)),
                  'היום': None,
                   'בעוד':Token.PLUS,
                  'לפני':Token.MINUS,
                  'חודשים':Token.MONTH,
                  'שבועות':Token.WEEK,
                  'ימים':Token.DAY,
                  'חודש':((Token.NUM, 2),(Token.MONTH, 0)),
                  'שבוע':((Token.NUM, 1),(Token.WEEK, 0)),
                  'יום':((Token.NUM, 2),(Token.DAY, 0)),
                  'חודשיים':((Token.NUM, 2),(Token.MONTH, 0)),
                  'שבועיים':((Token.NUM, 2),(Token.WEEK, 0)),
                  'יומיים':((Token.NUM, 2),(Token.DAY, 0)),
                  '.':Token.END}
        
        num_to_int={'אחד':1,
                  'שניים':2,
                  'שני': 2,
                   'שנים':2,
                  'שלושה':3,
                  'ארבעה':4,
                  'חמישה':5,
                  'שישה':6,
                  'שבעה':7,
                  'שמונה':8,
                  'תשעה':9,
                  'עשרה':10,
                  'עשר':10,
                  'עשרים':20,
                  'שלושים':30}
        
        parts=date_phrase[:-1].split()+['.']
        for p in parts:
            if p[0]=='ו':
                p = p[1:]
            if p in token_map:
                if type(token_map[p])==tuple:
                    tokens.extend(token_map[p])
                else:
                    tokens.append((token_map[p],0))
            else:
                tokens.append((Token.NUM, num_to_int[p]))

        return tokens
        
#         with open("date_grammar.txt",encoding='utf-8') as f:
#             for line in f:
#                 (key, val) = line.split()
#                 token_map[int(val)] = key

#         date_phrase = date_phrase[:-1].split()
#         tokens = []
#         for p in date_phrase:
#             tokens.append([key for key,val in token_map.items() if val == p][0])
#         return tokens


    def run(self, date_phrase):
        #return translated date in format yyyymmdd
        temp = self.tokenizer(date_phrase)
        temp = self.translate_date(temp)
        self.data =  self.data.format('YYYYMMDD')
        return self.data

In [185]:
data = ["לפני חמישה ימים.",
"לפני יומיים.",
"בעוד שלושה שבועות.",
"בעוד שבוע וארבעה ימים.",
"לפני שלושה חודשים וארבעה ימים.",
"לפני חודשיים ויום אחד."]

In [191]:
def main():
    #driver function
    for phrase in data:
        date_translator=DateTranslator('20200611')
        print(phrase)
        print(date_translator.run(phrase))
    #20200614

main()

לפני חמישה ימים.
20200606
לפני יומיים.
20200609
בעוד שלושה שבועות.
20200702
בעוד שבוע וארבעה ימים.
20200622
לפני שלושה חודשים וארבעה ימים.
20200307
לפני חודשיים ויום אחד.
20200409


In [142]:
data = arrow.now()
sign = '-'
time = '2'
print(sign+time)
i = int(sign+time)

dat = dt.strptime('20200611', "%Y%m%d")
dat = arrow.get(dat)
dat.shift(years=2)

-2


<Arrow [2022-06-11T00:00:00+00:00]>